### Simple GenAI App using Langchain

In [14]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPEN_AI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

In [15]:
### Data Ingestion- From the website scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [16]:
loader=WebBaseLoader("https://docs.langchain.com/langsmith/administration-overview")
loader

In [17]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='Overview - Docs by LangChainSkip to main contentLangGraph Platform is now part of LangSmith. Check out the Changelog for more information.Docs by LangChain home pageLangSmithSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationOverviewGet startedObservabilityEvaluationPrompt engineeringDeploymentHostingOverviewPlansAccount administrationOverviewCreate an account and API keySet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph Python SDKLangGraph JS/TS SDKLangSmith APILangGraph Server APIControl Plane API for LangSmith DeploymentAdditional resourcesFAQsRegions FAQPricing FAQAuthentication methodsReleases & changelogsData managementOn this pageResource HierarchyOrganizationsWorkspacesResou

In [8]:
### Load Data --> Docs --> Divide text into chunks --> text --> vectors --> Vector Enbeddings --> Vectir Store
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter =  RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents

[Document(metadata={'source': 'https://docs.langchain.com/langsmith/administration-overview', 'title': 'Overview - Docs by LangChain', 'language': 'en'}, page_content='Overview - Docs by LangChainSkip to main contentLangGraph Platform is now part of LangSmith. Check out the Changelog for more information.Docs by LangChain home pageLangSmithSearch...⌘KGitHubTry LangSmithTry LangSmithSearch...NavigationAccount administrationOverviewGet startedObservabilityEvaluationPrompt engineeringDeploymentHostingOverviewPlansAccount administrationOverviewCreate an account and API keySet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementReferenceLangSmith Python SDKLangSmith JS/TS SDKLangGraph SDK (Python)LangGraph SDK (JS/TS)LangSmith API referenceServer APIControl Plane APIRemoteGraphCLIEnvironment variablesAdditional resourcesFAQsRegions FAQPricing FAQAuthentication methodsReleases & changelogsData managementOn this pageResource HierarchyOrganizationsW

In [18]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [19]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents, embeddings)

In [20]:
vectorstoredb

In [ ]:
### Qwery from vector DB

qwery = "API key is no longer valid and cannot be reactivated or have its expiration modified."
result = vectorstoredb.similarity_search(qwery)
result[0].page_content

'\u200bAPI keys\nWe ended support for legacy API keys prefixed with ls__ on October 22, 2024 in favor of personal access tokens (PATs) and service keys. We require using PATs and service keys for all new integrations. API keys prefixed with ls__ will no longer work as of October 22, 2024.\n\u200bExpiration Dates\nWhen you create an API key, you have the option to set an expiration date. Adding an expiration date keys enhances security and minimize the risk of unauthorized access. For example, you may set expiration dates on keys for temporary tasks that require elevated access.\nBy default, keys never expire. Once expired, an API key is no longer valid and cannot be reactivated or have its expiration modified.\n\u200bPersonal Access Tokens (PATs)'

In [23]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001C97A74E630> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001C97A79C680> root_client=<openai.OpenAI object at 0x000001C97A74FE90> root_async_client=<openai.AsyncOpenAI object at 0x000001C97A79F7A0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********') stream_usage=True


In [24]:
### Retrival chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>

"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001C97A74E630>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001C97A79C680>, root_client=<openai.OpenAI object at 0x000001C97A74FE90>, root_async_client=<openai.AsyncOpenAI object at 0x000001C97A79F7A0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)
| StrOutputParser(),

In [25]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"API key is no longer valid and cannot be reactivated or have its expiration modified.",
    "context": [Document(page_content="""When you create an API key, you have the option to set an expiration date. Adding an expiration date keys enhances security and minimize the risk of unauthorized access. For example, you may set expiration dates on keys for temporary tasks that require elevated access.
By default, keys never expire. Once expired, an API key is no longer valid and cannot be reactivated or have its expiration modified. """)]
})

'Can an expired API key be reactivated or have its expiration date modified? \n\nNo, an expired API key cannot be reactivated or have its expiration date modified.'

In [27]:
### Input --> Retriver --> vectorstoredb

retriver = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain

retriver_chain = create_retrieval_chain(retriver,document_chain)
retriver_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001C976D0A780>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n'), additional_kwargs={})])
            | Ch

In [ ]:
response = retriver_chain.invoke({"input":"API key is no longer valid and cannot be reactivated or have its expiration modified."})
response[answer]